## モデルの作成～コンパイル～学習

In [ ]:
# import
import tensorflow as tf

# create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1]),
])
# summarize model
model.summary()
# compile model
model.compile(optimizer='sgd', loss='mean_squared_error')
# train model
model.fit("<train_data>", "<train_labels>", epochs=500)
# predict
print(model.predict([10.0]))
# evaluate
print(model.evaluate("<test_data>", "<test_labels>"))

#### layers
最初の層には`input_shape` (list or tupple) を指定する必要がある
* `tf.keras.layers.Dense(units, activation)` : 全結合層
* `tf.keras.layers.Flatten()` : フラット化層
* `tf.keras.layers.Conv2D(filters, kernel_size(tupple), activation)` : 畳み込み層
* `tf.keras.layers.MaxPooling2D(pool_size)` : プーリング層

#### activation
* "relu"
* "sigmoid"
* "softmax"

#### optimizer
* `tf.keras.optimizers.Adam(lr)` : Adamオプティマイザ
* `tf.keras.optimizers.SGD(lr)` : SGDオプティマイザ
* `tf.keras.optimizers.RMSprop(lr)` : RMSpropオプティマイザ
* `tf.keras.optimizers.Adagrad(lr)` : Adagradオプティマイザ
* `tf.keras.optimizers.Adadelta(lr)` : Adadeltaオプティマイザ

上記のオプティマイザを小文字で文字列として引数とすると、デフォルト設定で使用される

#### loss
* mean_squared_error
* sparse_categorical_crossentropy (one-hot encodingしていないとき)
* binary_crossentropy (one-hot encoding)
* categorical_crossentropy

#### metrics
* `tf.keras.metrics.Accuracy` : accuracy

## Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        threshold = 0.90
        if(logs.get("accuracy") >= threshold):
            print(f"Reached {threshold * 100}% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

model.fit("<train_data>", "<train_labels>", epochs=100, callbacks=[callbacks])

## Datasetのロード

In [ ]:
# fashion mnist
# train_images : (60000, 28, 28), 28x28のグレースケール画像
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# preprocess
train_images = train_images / 255.0
test_images = test_images / 255.0

## ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'horse-or-human/',  # source directory for training images
        target_size=(300, 300),  # All images will be resized to
        batch_size=128,
        class_mode='binary')

#### class_mode
* "categorical" : multi-class classification
* "binary" : binary classification

## Predict with CNN

In [ ]:
import numpy as np
from keras.preprocessing import image

# predicting images
path = "./uma.jpg"
img = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0])
if classes[0] > 0.5:
    print(" is a human")
else:
    print(" is a horse")